<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import csv
from keras import layers,Input
from keras.models import Model
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Stage 1 資料前處理

## 讀入向量化資料


In [2]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

In [3]:
len(data['1'])

764

## 讀入 True label csv

In [4]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/TrainLabel.csv'
response = urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
true_labels = list(csv.reader(lines))
del true_labels[0]

In [5]:
len(true_labels)

1381

## 建立 False label 資料

In [6]:
import random

input_keys = list(data.keys())
false_labels = []

while len(false_labels) < len(true_labels):
  temp = [random.choice(input_keys),random.choice(input_keys)]
  if temp not in true_labels:
    false_labels.append(temp)

In [7]:
len(false_labels)

1381

## 產生所需資料格式

In [8]:
def data_generator(input,Label=0):
  return [[[data[row[0]],data[row[1]]],Label] for row in input]

In [9]:
# 標籤為True的向量資料對
true_list = data_generator(true_labels,1)

In [10]:
# 標籤為False的向量資料對
false_list = data_generator(false_labels,0)

In [11]:
df_true = pd.DataFrame(true_list)
df_true.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [12]:
df_false = pd.DataFrame(false_list)
df_false.head()

,0,1
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0...",0


## 混合 Ture / False 標籤的資料

In [13]:
train_list = true_list+false_list

In [14]:
len(train_list)

2762

In [15]:
df_train = pd.DataFrame(train_list)
df_train.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [16]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)

In [17]:
df_train.head()

,0,1
2742,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0,...",0
1378,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",1
2470,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 2...",0
1714,"[[0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1979,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0


In [18]:
X = df_train[0]
Y = df_train[1]

## 區分訓練集與驗證集

In [19]:
# from sklearn.model_selection import train_test_split
# x_train, x_vaild, y_train, y_vaild = train_test_split(X, Y, test_size=.25)

In [20]:
# len(x_train)

In [21]:
# len(x_vaild)

### 轉為Numpy陣列

In [22]:
left_x = np.array([row[0] for row in X]).astype('float32')
right_x = np.array([row[1] for row in X]).astype('float32')
y_train = np.array(Y).astype('float32')

In [23]:
pd.DataFrame(left_x).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
pd.DataFrame(right_x).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
pd.DataFrame(y_train).head()

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0


# 建立神經網路模型

參考資料：
* [Image similarity estimation using a Siamese Network with a triplet loss](https://keras.io/examples/vision/siamese_network/)
* [Siamese Network — Unofficial Keras Implementation](https://github.com/gchoi/face-recognition-using-siamese-network)
* [Text-Similarity-Using-Siamese-Deep-Neural-Network](https://github.com/AnjaliDharmik/Text-Similarity-Using-Siamese-Deep-Neural-Network)

## 建構原理
藉由同一個層來一起進行萃取兩篇文章的相似度特徵  
因此需共享權重來訓練神經網路

In [37]:
lstm = layers.LSTM(382) # 共享權重的神經層

left_input = Input(shape=(764,),name='left_input')
# print(left_input.shape)
left_embedded_text = layers.Embedding(764,764)(left_input) # 輸入向量764；輸入向量764
left_output = lstm(left_embedded_text)
# print(left_output.shape)

right_input = Input(shape=(764,),name='right_input')
# print(right_input.shape)
right_embedded_text = layers.Embedding(764,764)(right_input)
right_output = lstm(right_embedded_text)
# print(right_output.shape)

merged = layers.concatenate([left_output,right_output],axis=-1) # 將向量串接
# print(merged.shape)
narrow_down_1 = layers.Dense(382,activation='relu')(merged)
narrow_down_2 = layers.Dense(191,activation='relu')(narrow_down_1)
narrow_down_3 = layers.Dense(96,activation='relu')(narrow_down_2)
narrow_down_4 = layers.Dense(48,activation='relu')(narrow_down_3)
predictions = layers.Dense(1,activation='sigmoid')(narrow_down_4)
model = Model([left_input,right_input],predictions)

In [38]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, 764)]        0           []                               
                                                                                                  
 right_input (InputLayer)       [(None, 764)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 764, 764)     583696      ['left_input[0][0]']             
                                                                                                  
 embedding_3 (Embedding)        (None, 764, 764)     583696      ['right_input[0][0]']            
                                                                                            

In [41]:
model = Model([left_input,right_input],predictions)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [42]:
# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'left_input':left_x,'right_input':right_x},y_train,epochs=10,batch_size=128,validation_split=0.25)

Epoch 1/10
17/17 [==============================] - 30s 2s/step - loss: 0.6992 - acc: 0.4824 - val_loss: 0.6974 - val_acc: 0.4964
Epoch 2/10
17/17 [==============================] - 25s 1s/step - loss: 0.6939 - acc: 0.4906 - val_loss: 0.6935 - val_acc: 0.4964
Epoch 3/10
17/17 [==============================] - 25s 1s/step - loss: 0.6948 - acc: 0.5031 - val_loss: 0.6936 - val_acc: 0.5036
Epoch 4/10
17/17 [==============================] - 25s 1s/step - loss: 0.6960 - acc: 0.4954 - val_loss: 0.6931 - val_acc: 0.5065
Epoch 5/10
17/17 [==============================] - 25s 1s/step - loss: 0.6932 - acc: 0.4993 - val_loss: 0.6935 - val_acc: 0.4964
Epoch 6/10
17/17 [==============================] - 25s 1s/step - loss: 0.6937 - acc: 0.4858 - val_loss: 0.6932 - val_acc: 0.5036
Epoch 7/10
17/17 [==============================] - 25s 1s/step - loss: 0.6956 - acc: 0.5041 - val_loss: 0.6931 - val_acc: 0.5036
Epoch 8/10
17/17 [==============================] - 25s 1s/step - loss: 0.6934 - acc: 0.50

# 載入Stage 2 比對資料


In [29]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage2/vectorize_data.json'
response = urlopen(url)
predict_data = json.loads(response.read())
df = pd.DataFrame(predict_data).T

In [30]:
known_assay = list(data.values())
compared_assay = list(predict_data.values())

In [31]:
len(known_assay)

559

In [ ]:
array = np.array(known_assay[0])
np.reshape(array, array.shape + (1,))

In [33]:
len(array)

764

### 錯誤解決
* [WARNING: WARNING:tensorflow:Model was constructed with shape (None, 150) , but it was called on an input with incompatible shape (None, 1)](https://stackoverflow.com/questions/61656444/warning-warningtensorflowmodel-was-constructed-with-shape-none-150-but-i)

In [44]:
# Test

for predict_row in compared_assay:
 
  for known_row in known_assay:
    predict_row = np.array(predict_row).astype('float32')
    known_row = np.array(known_row).astype('float32')
    np.reshape(predict_row, predict_row.shape + (1,))
    np.reshape(known_row, known_row.shape + (1,))
    print(predict_row.shape)
    print(known_row.shape)
    break
  break

(764,)
(764,)


In [45]:
results = {}
for predict_row in compared_assay:
  for known_row in known_assay:
    # temp = []
    predict_row = np.array(predict_row).astype('float32')
    known_row = np.array(known_row).astype('float32')
    np.reshape(predict_row, predict_row.shape + (1,))
    np.reshape(known_row, known_row.shape + (1,))
    print(predict_row.shape)
    print(known_row.shape)
    # prediction = model.predict({'left_input':predict_row,'right_input':known_row})
    # if prediction == 1:
      # temp.append(known_row)
    # results[predict_row] = known_row
    break
  break

(764,)
(764,)


In [ ]:
model.predict({'left_input':left_x,'right_input':right_x})